<a href="https://colab.research.google.com/github/thisaraniNJ/MachineLearning_CW/blob/main/MachineLearingCWbankfull.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
!pip install --upgrade imbalanced-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.4/238.4 kB 6.2 MB/s eta 0:00:00
  Attempting uninstall: imbalanced-learn
    Found existing installation: imbalanced-learn 0.12.4
    Uninstalling imbalanced-learn-0.12.4:
      Successfully uninstalled imbalanced-learn-0.12.4


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer


Load the dataset

In [2]:
# Load the dataset
df = pd.read_csv('/content/drive/MyDrive/ML CW/bank-full.csv', delimiter=";")

In [3]:
# Display the initial dataset structure
print("Dataset Overview: ")
print(df.info())

Dataset Overview: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45211 entries, 0 to 45210
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        45211 non-null  int64 
 1   job        45211 non-null  object
 2   marital    45211 non-null  object
 3   education  45211 non-null  object
 4   default    45211 non-null  object
 5   balance    45211 non-null  int64 
 6   housing    45211 non-null  object
 7   loan       45211 non-null  object
 8   contact    45211 non-null  object
 9   day        45211 non-null  int64 
 10  month      45211 non-null  object
 11  duration   45211 non-null  int64 
 12  campaign   45211 non-null  int64 
 13  pdays      45211 non-null  int64 
 14  previous   45211 non-null  int64 
 15  poutcome   45211 non-null  object
 16  y          45211 non-null  object
dtypes: int64(7), object(10)
memory usage: 5.9+ MB
None


In [4]:
print("\nFirst 5 rows of the dataset:")
print(df.head())


First 5 rows of the dataset:
   age           job  marital  education default  balance housing loan  \
0   58    management  married   tertiary      no     2143     yes   no   
1   44    technician   single  secondary      no       29     yes   no   
2   33  entrepreneur  married  secondary      no        2     yes  yes   
3   47   blue-collar  married    unknown      no     1506     yes   no   
4   33       unknown   single    unknown      no        1      no   no   

   contact  day month  duration  campaign  pdays  previous poutcome   y  
0  unknown    5   may       261         1     -1         0  unknown  no  
1  unknown    5   may       151         1     -1         0  unknown  no  
2  unknown    5   may        76         1     -1         0  unknown  no  
3  unknown    5   may        92         1     -1         0  unknown  no  
4  unknown    5   may       198         1     -1         0  unknown  no  


Encode categorical variables

In [5]:
# Encode the target variable ('y') to binary (yes: 1, no: 0)
df['y'] = df['y'].map({
    'yes': 1,
    'no':0
})

In [6]:
# Identify categorical columns
categorical_columns =[
    "job", "marital", "education", "default", "housing", "loan",
    "contact", "month", "poutcome"
]

In [7]:
# Encode categorical columns using LabelEncoder
encoder = LabelEncoder()
for col in categorical_columns:
    df[col] = encoder.fit_transform(df[col])

Handle Missing values

In [8]:
# Check for missing values
print("\nMissing values in the dataset: ")
print(df.isnull().sum())


Missing values in the dataset: 
age          0
job          0
marital      0
education    0
default      0
balance      0
housing      0
loan         0
contact      0
day          0
month        0
duration     0
campaign     0
pdays        0
previous     0
poutcome     0
y            0
dtype: int64


Feature Scaling

In [9]:
# Identify numerical columns
numerical_columns = [
    "age", "balance", "day","duration","campaign","pdays","previous"
]

In [10]:
# Scale numerical columns using StandardScaler
scaler = StandardScaler()
df[numerical_columns] = scaler.fit_transform(df[numerical_columns])

Handle class imbalance

In [11]:
# Separate features and target variable
X = df.drop('y', axis=1)
y = df['y']

In [12]:
# Handle class imbalance using SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

Train - Test split

In [13]:
X_train, X_test, y_train, y_test = train_test_split(
    X_resampled, y_resampled, test_size=0.3, random_state=42, stratify=y_resampled
)

In [14]:
# Display summary of the prepared dataset
print("\nTraining Dataset (X_train) Shape:", X_train.shape)
print("Test Dataset (X_test) Shape:", X_test.shape)
print("Training Target (y_train) Distribution:")
print(y_train.value_counts())
print("\nTest Target (y_test) Distribution:")
print(y_test.value_counts())


Training Dataset (X_train) Shape: (55890, 16)
Test Dataset (X_test) Shape: (23954, 16)
Training Target (y_train) Distribution:
y
0    27945
1    27945
Name: count, dtype: int64

Test Target (y_test) Distribution:
y
1    11977
0    11977
Name: count, dtype: int64


# Neural Network Model

Import necessary libraries

In [17]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

Define the neural network model

In [18]:
# Define the Neural Network Model
def build_nn_model(input_dim):
    model = Sequential([
        Input(shape=(input_dim,)),  # Define the input shape explicitly
        Dense(128, activation='relu'),
        Dropout(0.3),
        Dense(64, activation='relu'),
        Dropout(0.3),
        Dense(1, activation='sigmoid')  # Output layer for binary classification
    ])
    # Compile the model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model


In [19]:
# Initialize the model
input_dim = X_train.shape[1]  # Number of features
nn_model = build_nn_model(input_dim)

In [20]:
# Display model summary
print("\nNeural Network Model Summary:")
nn_model.summary()


Neural Network Model Summary:


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 128)                 │           2,176 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 10,497 (41.00 KB)

 Trainable params: 10,497 (41.00 KB)

 Non-trainable params: 0 (0.00 B)

Train the neural network model

In [23]:
batch_size = 8
epochs = 50

history = nn_model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    batch_size=batch_size,
    epochs=epochs,
    verbose=1
)

Epoch 1/50
6987/6987 ━━━━━━━━━━━━━━━━━━━━ 18s 3ms/step - accuracy: 0.8838 - loss: 0.2872 - val_accuracy: 0.8864 - val_loss: 0.2886
Epoch 2/50
6987/6987 ━━━━━━━━━━━━━━━━━━━━ 18s 3ms/step - accuracy: 0.8865 - loss: 0.2813 - val_accuracy: 0.8894 - val_loss: 0.2831
Epoch 3/50
6987/6987 ━━━━━━━━━━━━━━━━━━━━ 21s 3ms/step - accuracy: 0.8853 - loss: 0.2850 - val_accuracy: 0.8900 - val_loss: 0.2827
Epoch 4/50
6987/6987 ━━━━━━━━━━━━━━━━━━━━ 40s 3ms/step - accuracy: 0.8878 - loss: 0.2815 - val_accuracy: 0.8870 - val_loss: 0.2843
Epoch 5/50
6987/6987 ━━━━━━━━━━━━━━━━━━━━ 21s 3ms/step - accuracy: 0.8856 - loss: 0.2862 - val_accuracy: 0.8875 - val_loss: 0.2865
Epoch 6/50
6987/6987 ━━━━━━━━━━━━━━━━━━━━ 21s 3ms/step - accuracy: 0.8854 - loss: 0.2850 - val_accuracy: 0.8884 - val_loss: 0.2863
Epoch 7/50
6987/6987 ━━━━━━━━━━━━━━━━━━━━ 42s 3ms/step - accuracy: 0.8846 - loss: 0.2870 - val_accuracy: 0.8893 - val_loss: 0.2864
Epoch 8/50
6987/6987 ━━━━━━━━━━━━━━━━━━━━ 41s 3ms/step - accuracy: 0.8846 - loss: 0

Evaluate the Neural Network

In [24]:
print("\nEvaluating the Neural Network Model on Test Data:")
loss, accuracy = nn_model.evaluate(X_test, y_test, verbose=0)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")


Evaluating the Neural Network Model on Test Data:
Test Loss: 0.2871
Test Accuracy: 0.8880


Generate predictions and metrics

In [25]:
y_pred = (nn_model.predict(X_test) > 0.5).astype("int32")
print("\nClassification Report for Neural Network:")
print(classification_report(y_test, y_pred))

749/749 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step

Classification Report for Neural Network:
              precision    recall  f1-score   support

           0       0.93      0.84      0.88     11977
           1       0.85      0.94      0.89     11977

    accuracy                           0.89     23954
   macro avg       0.89      0.89      0.89     23954
weighted avg       0.89      0.89      0.89     23954



# Random Forest Classification

Import Libraries

In [26]:
from sklearn.ensemble import RandomForestClassifier
import numpy as np

Define and train the model

In [27]:
# Initialize the Random Forest model
rf_model = RandomForestClassifier(
    n_estimators=5000,  # Number of trees
    max_depth=None,    # No limit on depth of the trees
    random_state=42    # Ensure reproducibility
)

In [28]:
# Train the model
rf_model.fit(X_train, y_train)

RandomForestClassifier(n_estimators=5000, random_state=42)

Evaluate the random forest model

In [29]:
print("\nEvaluating the Random Forest Model on Test Data:")
y_pred_rf = rf_model.predict(X_test)

# Calculate accuracy
rf_accuracy = accuracy_score(y_test, y_pred_rf)
print(f"Random Forest Test Accuracy: {rf_accuracy:.4f}")

# Generate Classification Report
print("\nClassification Report for Random Forest:")
print(classification_report(y_test, y_pred_rf))


Evaluating the Random Forest Model on Test Data:
Random Forest Test Accuracy: 0.9362

Classification Report for Random Forest:
              precision    recall  f1-score   support

           0       0.96      0.91      0.93     11977
           1       0.92      0.96      0.94     11977

    accuracy                           0.94     23954
   macro avg       0.94      0.94      0.94     23954
weighted avg       0.94      0.94      0.94     23954



# Comparison of the Neural Network and Random Forest Classiffer

Import Necessary libraries